# MIT (TA1): From Paper and Code to annotated extraction


## 0. Preprocessing

In [1]:
import ast, json, requests, os
GPT_KEY = os.environ.get('OPENAI_API_KEY')
API_ROOT = "http://localhost:8000/"

## 1. MIT annotation end to end pipeline

#### We bring every annotation step together: for the original paper text, let's integrate all the extraction modules and output the MIT extraction:
[http://3.83.68.208/#/Paper-2-annotated-vars/upload_file_annotate_annotation_upload_file_extract__post](http://3.83.68.208/#/Paper-2-annotated-vars/upload_file_annotate_annotation_upload_file_extract__post)

In [3]:
scenario_1_paper = "text_ijerph-18-09027.txt"
with open(scenario_1_paper, 'rb') as f:
    files = {'file': ('filename', f)}
    params = {"gpt_key": GPT_KEY}
    response = requests.post(API_ROOT + "annotation/upload_file_extract/",  params=params,  files=files)
    json_str = response.text

In [4]:
print(json_str)
### TOO LARGE ! 

{"attributes":[{"type":"anchored_extraction","amr_element_id":null,"payload":{"id":{"id":"mit0"},"names":[{"id":{"id":"mit0"},"name":"γ","extraction_source":null,"provenance":{"method":"MIT extractor V1.0 - text, dataset, formula annotation (chunwei@mit.edu)","timestamp":"2023-10-25T21:02:26.688528"}}],"descriptions":[{"id":{"id":"mit0"},"source":"Proportion of recovery per day","grounding":null,"extraction_source":null,"provenance":{"method":"MIT extractor V1.0 - text, dataset, formula annotation (chunwei@mit.edu)","timestamp":"2023-10-25T21:02:26.688528"}}],"value_specs":[{"id":{"id":"mit0-value"},"value":{"source":"1/11","grounding":null,"extraction_source":null},"units":null,"type":null,"bounds":null,"provenance":{"method":"MIT extractor V1.0 - text, dataset, formula annotation (chunwei@mit.edu)","timestamp":"2023-10-25T21:02:26.688528"}}],"groundings":[{"grounding_text":"incomplete AE recovery","grounding_id":"oae:0001873","source":[],"score":1.0,"provenance":{"method":"MIT extrac

In [5]:
import ta1viewer # SAME REPO AS EXTRACTION VIEWER
from ta1viewer import display_json, display_pdf_bbox 
#pip install git+https://github.com/orm011/ta1viewer.git@main
display_json(json_str)

In [6]:
import json

json_object = json.dumps(json.loads(json_str), indent=4)
with open("m_temp_scenario_1.json", "w") as f:
    f.write(json_object)

## 2. Interacting with the University of Arizona extraction

#### With both the University of Arizona and MIT extractions, we first build the entity matching mapping for all the extracted variable entities, and then integrate the two integration together with the unified TA1 data model.
[http://3.83.68.208/#/TA1-Integration/upload_files_integration_integration_get_mapping_post](http://3.83.68.208/#/TA1-Integration/upload_files_integration_integration_get_mapping_post)

In [7]:
with open('m_temp_scenario_1.json', 'rb') as f_mit, open('ijerph-18-09027.json', 'rb') as f_arizona:
    files = { 'mit_file': ('bucky__mit-extraction_id.json', f_mit, 'application/json'),
        'arizona_file': ('bucky_arizona_output_example.json', f_arizona, 'application/json')}
    params = {"gpt_key": GPT_KEY}
    response = requests.post(API_ROOT + "integration/get_mapping", params=params, files=files)
    print(response.text)

{"attributes":[{"type":"anchored_extraction","amr_element_id":null,"payload":{"id":{"id":"E:-1621000196"},"names":[{"id":{"id":"T:-2105346093"},"name":"potential prevention of up","extraction_source":{"page":0,"block":0,"char_start":524,"char_end":550,"document_reference":{"id":"ijerph-18-09027.pdf"}},"provenance":{"method":"Skema TR Pipeline rules","timestamp":"2023-07-04T01:15:56.887148"}}],"descriptions":[],"value_specs":[{"id":{"id":"T:-202544802"},"value":{"source":"%","grounding":[],"extraction_source":{"page":0,"block":0,"char_start":556,"char_end":557,"document_reference":{"id":"ijerph-18-09027.pdf"}}},"units":null,"type":null,"bounds":null,"provenance":{"method":"Skema TR Pipeline rules","timestamp":"2023-07-04T01:15:56.887148"}}],"groundings":[{"grounding_text":"Centers for Disease Control and Prevention","grounding_id":"ncit:C16408","source":[],"score":0.7546381950378418,"provenance":{"method":"SKEMA-TR-Embedding","timestamp":"2023-07-04T01:15:56.887183"}}],"data_columns":nu

In [8]:
display_json(response.text)